# SCRIPT DE FRAGMENTACIÓN DE IMÁGENES (TILING)

Divide imágenes satelitales grandes y sus máscaras correspondientes en recortes (patches) más pequeños de tamaño fijo (ej. 256x256). Esto es necesario porque las redes neuronales como U-Net requieren entradas de tamaño constante y moderado para no saturar la memoria VRAM.


In [ ]:
import os
from PIL import Image

# === CONFIGURACIÓN DE DIRECTORIOS ===
# Directorios de entrada (Imágenes ya aumentadas o originales)
INPUT_IMAGES_DIR = "Images"
INPUT_MASKS_DIR = "Masks"

# Directorios de salida (Donde se guardarán los recortes)
OUTPUT_IMAGES_DIR = "Tiled_Images/" # Sugerencia: renombrar a su carpeta principal de imagenes
OUTPUT_MASKS_DIR = "Tiled_Masks/" # Sugerencia: renombrar a su carpeta principal de máscaras

# Crear directorios si no existen
os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)
os.makedirs(OUTPUT_MASKS_DIR, exist_ok=True)

# Tamaño del recorte (Debe coincidir con la entrada de la U-Net)
PATCH_SIZE = 256

def crop_image(img, patch_size=256):
    """
    Recorta una imagen en parches secuenciales de tamaño fijo.
    
    Args:
        img (PIL.Image): Objeto de imagen abierto con Pillow.
        patch_size (int): Tamaño del lado del cuadrado (ej. 256).
        
    Returns:
        list: Lista de tuplas [((x, y), recorte)].
    """
    w, h = img.size
    crops = []
    
    # Recorrer la imagen en pasos del tamaño del parche (Stride = Patch Size)
    # Esto genera recortes adyacentes sin superposición.
    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            
            # Definir coordenadas de la caja de recorte (left, upper, right, lower)
            box = (x, y, x + patch_size, y + patch_size)
            crop = img.crop(box)
            
            # --- CRITERIO DE ACEPTACIÓN ---
            # Solo guardamos el recorte si tiene el tamaño EXACTO (256x256).
            # Esto descarta los bordes derechos e inferiores si la imagen original
            # no es múltiplo exacto de 256. Esto es preferible en entrenamiento
            # para evitar introducir tensores de tamaño incorrecto o con relleno (padding)
            # que podría confundir a la red.
            if crop.size == (patch_size, patch_size):
                crops.append(((x, y), crop))
                
    return crops

# === PREPARACIÓN ===
# Crear diccionario de máscaras para búsqueda rápida
# Clave: nombre del archivo sin extensión (en minúsculas) -> Valor: nombre real del archivo
mask_dict = {}
for m in os.listdir(INPUT_MASKS_DIR):
    base = os.path.splitext(m)[0].lower()
    mask_dict[base] = m

print(f"Iniciando proceso de Tiling (Patch Size: {PATCH_SIZE}x{PATCH_SIZE})...")

# === PROCESAMIENTO PRINCIPAL ===
for img_file in os.listdir(INPUT_IMAGES_DIR):

In [ ]:
import os
from PIL import Image

# Rutas
input_images = "images_principales"
input_masks = "mask_binarias"

output_images = "recortes_imagenes_256/"
output_masks = "recortes_mascaras_256/"

# Crear carpetas si no existen
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_masks, exist_ok=True)

PATCH_SIZE = 256

def crop_image(img, patch_size=256):
    w, h = img.size
    crops = []

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            box = (x, y, x + patch_size, y + patch_size)
            crop = img.crop(box)

            # Solo guardar si el recorte tiene tamaño exacto
            if crop.size == (patch_size, patch_size):
                crops.append(((x, y), crop))

    return crops

# Procesar cada imagen y su máscara
for filename in os.listdir(input_images):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff")):

        img_path = os.path.join(input_images, filename)
        mask_path = os.path.join(input_masks, filename)

        # Asegurar que exista la máscara correspondiente
        if not os.path.exists(mask_path):
            print(f"No se encontró máscara para {filename}, omitido.")
            continue

        # Abrir imagen y máscara
        img = Image.open(img_path)
        mask = Image.open(mask_path)

        # Recortar
        img_crops = crop_image(img, PATCH_SIZE)
        mask_crops = crop_image(mask, PATCH_SIZE)

        # Deben tener la misma cantidad de parches
        if len(img_crops) != len(mask_crops):
            print(f"Advertencia: {filename} tiene distinto número de parches.")
            continue

        # Guardar los recortes
        for i, ((pos_img, crop_img), (pos_mask, crop_mask)) in enumerate(zip(img_crops, mask_crops)):
            x, y = pos_img
            base = filename.rsplit(".", 1)[0]

            img_name = f"{base}_{x}_{y}.jpg"
            mask_name = f"{base}_{x}_{y}.png"

            crop_img.save(os.path.join(output_images, img_name))
            crop_mask.save(os.path.join(output_masks, mask_name))

        print(f"✔ Recortado: {filename}")
